In [1]:
%matplotlib inline

# Aprender Pytorch con ejemplos - Autograd

En los ejemplos anteriores, tuvimos que implementar manualmente los pases hacia adelante y hacia atrás de nuestra red neuronal. La implementación manual del paso hacia atrás no es un gran problema para una red pequeña de dos capas, pero puede complicarse rápidamente en redes grandes y complejas.

Afortunadamente, podemos usar [la diferenciación automática](https://en.wikipedia.org/wiki/Automatic_differentiation) para automatizar el cálculo de pases hacia atrás en redes neuronales. El paquete autograd en PyTorch proporciona exactamente esta funcionalidad. Al usar autograd, el pase directo de su red definirá un gráfico computacional; los nodos en el gráfico serán tensores y los bordes serán funciones que producen tensores de salida a partir de tensores de entrada. La retropropagación a través de este gráfico le permite calcular gradientes fácilmente.

Esto suena complicado, es bastante sencillo de usar en la práctica. Cada tensor representa un nodo en un gráfico computacional. Si ``x`` es un Tensor que tiene, ``x.requires_grad=True`` entonces ``x.grad`` es otro Tensor que mantiene el gradiente de ``x`` con respecto a algún valor escalar.

Aquí usamos PyTorch Tensors y autograd para implementar nuestro ejemplo de onda sinusoidal de ajuste con polinomio de tercer orden; ahora ya no necesitamos implementar manualmente el paso hacia atrás a través de la red:

In [2]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 83.7930679321289
199 58.96221923828125
299 42.38225555419922
399 31.299880981445312
499 23.88750648498535
599 18.926542282104492
699 15.603999137878418
799 13.377126693725586
899 11.883524894714355
999 10.88094425201416
1099 10.207419395446777
1199 9.7545747756958
1299 9.449835777282715
1399 9.244585037231445
1499 9.106199264526367
1599 9.012828826904297
1699 8.949758529663086
1799 8.907115936279297
1899 8.878255844116211
1999 8.858701705932617
Result: y = 0.004409832414239645 + 0.8519610166549683 x + -0.0007607686566188931 x^2 + -0.09265050292015076 x^3
